In [1]:
import pandas as pd
import pubchempy as pcp
import os

In [3]:
rec_lig_df = pd.read_csv("IUPHAR/interactions.csv", skiprows=[0])
rec_lig_df = rec_lig_df[['Target', 'Target ID', 'Ligand', 'Ligand ID']].drop_duplicates().dropna()
rec_lig_df

/tmp/ipykernel_2361482/1604033830.py:1: DtypeWarning: Columns (6,8,9,10,11,39) have mixed types. Specify dtype option on import or set low_memory=False.
  rec_lig_df = pd.read_csv("IUPHAR/interactions.csv", skiprows=[0])


,Target,Target ID,Ligand,Ligand ID
0,12S-LOX,1387.0,ML355,8752
1,15-LOX-1,1388.0,PKUMDL_MH_1001,9141
2,15-LOX-1,1388.0,compound 34 [PMID: 20866075],8751
3,15-LOX-1,1388.0,ML351,10263
4,15-LOX-1,1388.0,PD-146176,11390
...,...,...,...,...
22611,zeta chain of T cell receptor associated prote...,2285.0,aloisine,5923
22612,zeta chain of T cell receptor associated prote...,2285.0,mivavotinib,9600
22613,zeta chain of T cell receptor associated prote...,2285.0,Syk inhibitor II,6047
22614,Zika virus genome polyprotein,2953.0,NSC157058,9652


In [12]:
def contains_invalid_chars(smiles):
    invalid_chars = ['(', ')', '<', '>']
    return any(char in smiles for char in invalid_chars)

TRPV_ligands = rec_lig_df[rec_lig_df['Target ID']==507.0].drop_duplicates()
TRPV_ligands = TRPV_ligands[~TRPV_ligands['Ligand'].apply(contains_invalid_chars)]
TRPV_ligands = TRPV_ligands['Ligand'].tolist()
TRPV_ligands

['anandamide',
 'DkTx',
 'RhTx',
 'zucapsaicin',
 '12S-HPETE',
 '2-APB',
 '5S-HPETE',
 'allicin',
 'arvanil',
 'capsaicin',
 'olvanil',
 'piperine',
 'PPAHV',
 'resiniferatoxin',
 'vanillotoxin-1',
 'vanillotoxin-2',
 'vanillotoxin-3',
 '5S-HETE',
 'camphor',
 'diphenylboronic anhydride',
 'phenylacetylrinvanil',
 'MRS-1477',
 's-RhTx',
 'asivatrep',
 'A-1165442',
 'BCTC',
 'capsazepine',
 'JNJ17203212',
 'JYL1421',
 'mavatrep',
 'ruthenium red',
 'SB452533',
 'SB705498',
 'agatoxin 489',
 'AMG 9810',
 "5'-iodoresiniferatoxin",
 '6-iodo-nordihydrocapsaicin',
 'A425619',
 'A778317',
 'AMG517',
 'AMG628',
 'NADA',
 'SB366791',
 'AZD1386']

In [4]:
#name2pdb 전체 적용
import utils

os.makedirs(os.path.join(os.getcwd(), 'IUPAHR_dock'), exist_ok= True)
for lig in TRPV_ligands['Ligand'].tolist():
    utils.name2pdb(lig, os.path.join(os.getcwd(), 'IUPAHR_dock'))

Converted anandamide to /home/kjs/Downloads/AutoDockTools_py3/pycode/IUPAHR_dock/anandamide.pdb
Not found: DkTx (404 error)
Not found: RhTx (404 error)
Converted zucapsaicin to /home/kjs/Downloads/AutoDockTools_py3/pycode/IUPAHR_dock/zucapsaicin.pdb
Converted 12S-HPETE to /home/kjs/Downloads/AutoDockTools_py3/pycode/IUPAHR_dock/12S-HPETE.pdb
Converted 2-APB to /home/kjs/Downloads/AutoDockTools_py3/pycode/IUPAHR_dock/2-APB.pdb
Converted 5S-HPETE to /home/kjs/Downloads/AutoDockTools_py3/pycode/IUPAHR_dock/5S-HPETE.pdb
Converted allicin to /home/kjs/Downloads/AutoDockTools_py3/pycode/IUPAHR_dock/allicin.pdb
Converted arvanil to /home/kjs/Downloads/AutoDockTools_py3/pycode/IUPAHR_dock/arvanil.pdb
Converted capsaicin to /home/kjs/Downloads/AutoDockTools_py3/pycode/IUPAHR_dock/capsaicin.pdb
Converted olvanil to /home/kjs/Downloads/AutoDockTools_py3/pycode/IUPAHR_dock/olvanil.pdb
Converted piperine to /home/kjs/Downloads/AutoDockTools_py3/pycode/IUPAHR_dock/piperine.pdb
Converted PPAHV to /ho

In [13]:
import subprocess
    
receptor = "apo_8gfa.pdbqt"
ligands = TRPV_ligands

def run_dock(receptor, ligand):
    try:
        logging.info(f"Processing ligand: {ligand}")
        subprocess.run(["python3", "./dock.py", "-r", receptor, "-l", ligand, "-o", "dockpy_res"], check=True)
        logging.info(f"Completed ligand: {ligand}")
    except subprocess.CalledProcessError as e:
        logging.error(f"Error processing ligand: {ligand}, Error: {e}")

for lig in ligands:
    run_dock(receptor, lig)

2024-06-15 15:01:06,935 - INFO - Processing ligand: anandamide
2024-06-15 15:01:36,602 - INFO - Completed ligand: anandamide
2024-06-15 15:01:36,603 - INFO - Processing ligand: DkTx


#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, Journal of Computational Chemistry 31 (2010)  #
# 455-461                                                       #
#                                                               #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see http://vina.scripps.edu for more information.      #
#################################################################

Detected 16 CPUs
Reading input ... done.
Setting up the scoring function ... done.
Analyzing the binding site ... done.
Using random seed: 20

2024-06-15 15:01:37,684 - ERROR - Error processing ligand: DkTx, Error: Command '['python3', './dock.py', '-r', 'apo_8gfa.pdbqt', '-l', 'DkTx', '-o', 'dockpy_res']' returned non-zero exit status 1.
2024-06-15 15:01:37,685 - INFO - Processing ligand: RhTx


Not found: DkTx (404 error)
Error: Converting DkTx pdb file failed, quitting dock


2024-06-15 15:01:38,802 - ERROR - Error processing ligand: RhTx, Error: Command '['python3', './dock.py', '-r', 'apo_8gfa.pdbqt', '-l', 'RhTx', '-o', 'dockpy_res']' returned non-zero exit status 1.
2024-06-15 15:01:38,803 - INFO - Processing ligand: zucapsaicin


...

#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, Journal of Computational Chemistry 31 (2010)  #
# 455-461                                                       #
#                                                               #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see http://vina.scripps.edu for more information.      #
#################################################################

Detected 16 CPUs
Reading input ... done.
Setting up the scoring function ... done.
Analyzing the binding site ... done.
Using random seed: -6

2024-06-15 15:13:22,338 - INFO - Completed ligand: SB366791
2024-06-15 15:13:22,339 - INFO - Processing ligand: AZD1386


#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, Journal of Computational Chemistry 31 (2010)  #
# 455-461                                                       #
#                                                               #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see http://vina.scripps.edu for more information.      #
#################################################################

Detected 16 CPUs
Reading input ... done.
Setting up the scoring function ... done.
Analyzing the binding site ... done.
Using random seed: 12

2024-06-15 15:13:34,408 - INFO - Completed ligand: AZD1386


#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, Journal of Computational Chemistry 31 (2010)  #
# 455-461                                                       #
#                                                               #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see http://vina.scripps.edu for more information.      #
#################################################################

Detected 16 CPUs
Reading input ... done.
Setting up the scoring function ... done.
Analyzing the binding site ... done.
Using random seed: 52